### Install Prerequisite

In [ ]:
!pip install gradio jinja2

### Importing Libraries

In [1]:
import os
import numpy as np
import pandas as pd
import sys
sys.path.append('C:/Users/JOY/Desktop/Final Project')

from sklearn.model_selection import train_test_split

from keras import backend as K
import tensorflow as tf
import tf.keras.preprocessing.text as tk
import tf.keras.preprocessing.sequence as ps
from tensorflow.keras.layers import Input, Dropout, Bidirectional, Dense, Embedding, Conv1D
from tensorflow.keras.layers import GlobalAveragePooling1D, GlobalMaxPooling1D, concatenate

import Caribe as cb
import gradio as gr

### Data Loading

In [20]:
df = pd.read_csv(os.path.join('datasets/jigsaw-toxic-comment-classification-challenge','train.csv', 'train.csv'))
GLOVE_EMBEDDING = "datasets/jigsaw-toxic-comment-classification-challenge/glove.6B.100d.txt"

column_names=["short","long"]
df1 = pd.read_csv("datasets/abbrevations.csv",names=column_names)
df2 = pd.read_csv("datasets/emoji_df.csv")
df3 = pd.read_csv("datasets/twitterSlang.csv")

39130     "\n\n happy birthday! \n\nhey,  .  just stoppi...
25049     i have no problem with the homosexual organisa...
437       i never fucking made this mother fucking artic...
93449     sonic the hedgehog \n\nsonic the hedgehog (ソニッ...
35166     later, due to a corruption of the meaning, bot...
                                ...                        
73349     who is in charge of russia? putin or medevev? ...
109259                        on edit: forgot to actually .
50057                     "\ncopying to talk page. serenc "
5192                re-adding anti-barnstar\n anti-barnstar
128037    fuck wikipedia!!! \n\npeace (except to wikipedia)
Name: comment_text, Length: 111699, dtype: object


### Train Test Split

In [ ]:
train, test = train_test_split(df, test_size = 0.3, random_state = 1)

print("Train:", train.shape)
print("Test:", test.shape)

In [ ]:
train["comment_text"].fillna("fillna")
test["comment_text"].fillna("fillna")

x_train = train["comment_text"].str.lower()
x_test = test["comment_text"].str.lower()

y_train = train[["Toxic", "Severe Toxic", "Obscene", "Threat", "Insult", "Identity Hate"]].values
y_test = test[["Toxic", "Severe Toxic", "Obscene", "Threat", "Insult", "Identity Hate"]].values

### Tokenizer

In [ ]:
max_words = 100000
max_len = 150

embed_size = 100

tokenizer = tk.Tokenizer(num_words=max_words, lower=True)

tokenizer.fit_on_texts(x_train)

x_train = tokenizer.texts_to_sequences(x_train)
x_test = tokenizer.texts_to_sequences(x_test)

x_train = ps.pad_sequences(x_train, maxlen=max_len)
x_test = ps.pad_sequences(x_test, maxlen=max_len)

### Glove Embedding Vectorizer

In [3]:
embeddings_index = {}

with open(GLOVE_EMBEDDING, encoding='utf8') as f:
    for line in f:
        values = line.rstrip().rsplit(' ')
        word = values[0]
        embed = np.asarray(values[1:], dtype='float32')
        embeddings_index[word] = embed

word_index = tokenizer.word_index

num_words = min(max_words, len(word_index) + 1)

embedding_matrix = np.zeros((num_words, embed_size), dtype='float32')

for word, i in word_index.items():

    if i >= max_words:
        continue

    embedding_vector = embeddings_index.get(word)

    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector

### Precision Recall F1

In [33]:
def recall_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall

def precision_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision

def f1_m(y_true, y_pred):
    precision = precision_m(y_true, y_pred)
    recall = recall_m(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))

### Defining CNN Model

In [34]:
input = Input(shape=(max_len,))

x = Embedding(max_words, embed_size, weights=[embedding_matrix], trainable=False)(input)

x = Bidirectional(tf.keras.layers.GRU(128, return_sequences=True, dropout=0.1,
                                                      recurrent_dropout=0.1))(x)

x = Conv1D(64, kernel_size=3, padding="valid", kernel_initializer="glorot_uniform")(x)

avg_pool = GlobalAveragePooling1D()(x)
max_pool = GlobalMaxPooling1D()(x)

x = concatenate([avg_pool, max_pool])

preds = Dense(6, activation="sigmoid")(x)

model = tf.keras.Model(input, preds)

### Summarize Model

In [35]:
model.summary()
model.compile(loss='binary_crossentropy', optimizer=tf.keras.optimizers.Adam(lr=1e-3), metrics=['acc',f1_m,precision_m, recall_m])

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_2 (InputLayer)           [(None, 150)]        0           []                               
                                                                                                  
 embedding_1 (Embedding)        (None, 150, 100)     10000000    ['input_2[0][0]']                
                                                                                                  
 bidirectional_1 (Bidirectional  (None, 150, 256)    176640      ['embedding_1[0][0]']            
 )                                                                                                
                                                                                                  
 conv1d_1 (Conv1D)              (None, 148, 64)      49216       ['bidirectional_1[0][0]']  

J:\Program Files\Anaconda3\envs\python37\lib\site-packages\keras\optimizer_v2\adam.py:105: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


### Train Model

In [36]:
batch_size = 128

checkpoint_path = "training_1/cp.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)

cp_callback = tf.keras.callbacks.ModelCheckpoint(checkpoint_path,save_weights_only=True,verbose=1)

callbacks = [tf.keras.callbacks.EarlyStopping(patience=5, monitor='val_loss'),tf.keras.callbacks.TensorBoard(log_dir='./logs'),cp_callback]

model.fit(x_train, y_train, validation_split=0.2, batch_size=batch_size, epochs=3, callbacks=callbacks, verbose=1)

Epoch 1/3
699/699 [==============================] - ETA: 0s - loss: 0.0694 - acc: 0.9110 - f1_m: 0.5944 - precision_m: 0.7433 - recall_m: 0.5188
Epoch 00001: saving model to training_1\cp.ckpt
699/699 [==============================] - 812s 1s/step - loss: 0.0694 - acc: 0.9110 - f1_m: 0.5944 - precision_m: 0.7433 - recall_m: 0.5188 - val_loss: 0.0584 - val_acc: 0.9770 - val_f1_m: 0.7029 - val_precision_m: 0.7111 - val_recall_m: 0.7053
Epoch 2/3
699/699 [==============================] - ETA: 0s - loss: 0.0516 - acc: 0.9330 - f1_m: 0.6988 - precision_m: 0.8097 - recall_m: 0.6294
Epoch 00002: saving model to training_1\cp.ckpt
699/699 [==============================] - 805s 1s/step - loss: 0.0516 - acc: 0.9330 - f1_m: 0.6988 - precision_m: 0.8097 - recall_m: 0.6294 - val_loss: 0.0495 - val_acc: 0.9822 - val_f1_m: 0.7075 - val_precision_m: 0.7984 - val_recall_m: 0.6448
Epoch 3/3
699/699 [==============================] - ETA: 0s - loss: 0.0465 - acc: 0.9387 - f1_m: 0.7280 - precision_m: 

### Load the Latest Model

In [37]:
latest = tf.train.latest_checkpoint(checkpoint_dir)
model.load_weights(latest)

<class 'numpy.ndarray'>
[  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0  23 997 129  47   6
 121   2 438  30  23  94]
['article creation where would i go to start an article thanks']
[0 0 0 0 0 0]
[[6.3482631e-04 5.3460582e-05 2.8034404e-04 4.6893547e-05 5.3904607e-04
  4.4846576e-05]]


### Model Predictions

In [ ]:
predictions = model.predict(np.expand_dims(x_train[43], 0))

print(tokenizer.sequences_to_texts([x_train[43]]))
print(y_train[43])
print(predictions)

### Model Evaluation

In [38]:
loss, accuracy, f1_score, precision, recall = model.evaluate(x_test, y_test, verbose=1)

1496/1496 [==============================] - 226s 151ms/step - loss: 0.0494 - acc: 0.8488 - f1_m: 0.6401 - precision_m: 0.7228 - recall_m: 0.6144


In [39]:
print(f'Precision: {precision}, Recall:{recall}, Accuracy:{accuracy}, F1:{f1_score}')

Precision: 0.7227891683578491, Recall:0.6144306063652039, Accuracy:0.8487842679023743, F1:0.6400859355926514


### Save the Model 

In [40]:
model.save('models/rnn03.h5')

### Load the Model

In [43]:
model = tf.keras.models.load_model('models/rnn03.h5', custom_objects={"f1_score": f1_m, "precision": precision_m, "recall": recall_m})

ValueError: Unknown metric function: f1_m. Please ensure this object is passed to the `custom_objects` argument. See https://www.tensorflow.org/guide/keras/save_and_serialize#registering_the_custom_object for details.

### Pre-Processing Filters

In [ ]:
def preprocessingfilters(comment):
    print("\nOriginal sentence : " ,comment)

    #Slang 
    for i in df3.loc[:, 'slang']:
        comment = comment.lower()
        if(i.lower() in comment.split(" ")):
            j = df3[i==df3['slang']]
            k = j.iloc[-1].values

        comment = comment.lower()
        comment = comment.replace(i.lower(), k[1])
        print("Slang Expanded sentence : " ,comment)

    # Abberivation
    for i in df1.loc[:, 'short']:
        comment = comment.lower()
        if(i.lower() in comment.split(" ")):
            j = df1[i==df1['short']]
            k = j.iloc[-1].values

        comment = comment.lower()
        comment = comment.replace(i.lower(), k[1])
        print("Abberivation Expanded sentence : " ,comment)

    # Emojis
    for i in df2.loc[:, 'emoji']:
        if(i in comment):
            j = df2[i==df2['emoji']]
            k = j.iloc[-1].values

        comment = comment.replace(i, ", having " +k[1])
        print("Emojis Expanded sentence : " ,comment)

    comment=cb.caribe_corrector(comment)
    print("Correct sentence : " ,comment)
    return comment

### User Interface

In [ ]:
#Importing detoxification function from detox file
from detox import detoxification

### UI Action Function

In [ ]:
def score_comment(comment):
    fcomment = preprocessingfilters(comment)
    comment1 = tokenizer.texts_to_sequences([comment])
    comment1 = tf.keras.preprocessing.sequence.pad_sequences(comment1, maxlen=max_len)
    
    results = model.predict(np.expand_dims(comment1[0], 0))
    
    if results[0][0] > 0.1:
        pred = detoxification(comment)
        pred = (str(pred)[1:-1]).strip('[\]",')
        pred = str(pred)
    else:
        pred = "Non Toxic Sentence" 

    d = []
    for idx, col in enumerate(df.columns[2:]):
        d.append(
            {
                'Catagory': col,
                'Result': results[0][idx] > 0.1,
                'Percent':  round((results[0][idx])*100,2)
            }
        )

    d = pd.DataFrame(d)      
        
    return fcomment,d,pred

### UI Modeling

In [ ]:
with gr.Blocks() as interface:
    name = gr.inputs.Textbox(lines=2, placeholder='Enter Your Sentence', label = "Input Sentence")
    greet_btn = gr.Button("Submit")
    output = [gr.Textbox(label="Did you mean?"),gr.Dataframe(label="Toxicity Detection & Classification",headers=['Catagory', 'Result', 'Percent']) , gr.Textbox(label="Detoxification")]
    greet_btn.click(fn=score_comment, inputs=name, outputs=output)

### UI Launch

In [ ]:
interface.launch(share=True)